<a href="https://colab.research.google.com/github/cristinarainich/Sternberg_analysis/blob/main/Creating_data_frames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [287]:
import pandas as pd
import numpy as np
import pylab as pl
import scipy.io
from pathlib import Path

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [289]:
working_path = Path('/content/gdrive/MyDrive/CoganLab')

In [290]:
# controls data
eeg = scipy.io.loadmat(
    working_path.joinpath('Subject_NeighborhoodSternberg_EEG_060322.mat')
    )
# patients data
ieeg = scipy.io.loadmat(
    working_path.joinpath('Subject_NeighborhoodSternberg_IEEG_060322.mat')
    )

## iEEG dataset 

In [329]:
data_ieeg = ieeg['Subject'] # contains trials for each of the subjects
print('Number of subjects in EEG dataset is', data_ieeg.shape[1])

Number of subjects in EEG dataset is 30


In [377]:
subjects_ieeg = [] # subject numbers
for i in data_ieeg['Name'][0]:
  subjects_ieeg.append(i[0])

In [420]:
def get_to_df(data):
  # this function reads into .mat file and creates df out of it for all of the trials of one subject
  # i'm planning on making it more beautiful later

  columns = data[0][1].dtype.names
  df = pd.DataFrame(columns=columns)
  df['Trial'] = np.nan
  df_ = df.copy()
  for i in range(0, data.shape[1]): # for each trial number
    mdtype = data[0][i].dtype
    values = []
    for name in mdtype.names:
      values.append(data[0][i][name])
    values = extra_arrays(values)
    values.append(str(i)) # adding a trial number at the end
    df_.loc[len(df_.index)] = values
  return df_

In [444]:
def extra_arrays(x):
  # elements have nested structure, i.e. arrays inside the arrays
  # below we're getting rid of this depending on the shape of values

  values = []
  for elem in range(0, len(x)):
    var = x[elem]
    var = var[0, 0]
    if var.shape == (1, 1):
      var = var[0, 0]
      values.append(var)
    elif var.shape == (1,):
      values.append(var[0])
    elif var.shape == (1, 5):
      var = var.tolist()[0]
      values.append(var)
    elif var.shape == (5, 1):
      new = []
      for i in var:
        i = i[0]
        new.append(i)
      values.append(new)
    else:
      values.append(var)
  return values

In [445]:
columns_ieeg = data_ieeg['trialInfo'][0, 0][0, 0].dtype.names # name of the variables from trial info
final_ieeg = pd.DataFrame(columns=columns_ieeg)
final_ieeg['Trial'] = np.nan
final_ieeg['Subject'] = np.nan
for subj in range(0, data_ieeg['trialInfo'].shape[1]): # for each subject
  print(subj)
  subj_df = get_to_df(data_ieeg['trialInfo'][0, subj])
  subj_df['Subject'] = subjects_ieeg[subj]
  final_ieeg = pd.concat([final_ieeg, subj_df], axis = 0)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [446]:
final_ieeg.head()

,block,ProbeType,ProbeTypeName,StimlusCategory,ReactionTime,KeyCode,Resp,RespCorrect,Omission,stimulusAudioStart,...,stimulusSounds_idx,stimulusSounds_name,ProbeCategory,probeSound_idx,probeSound_name,Trial,Subject,ListenCueTime,MaintenancePeriodTime,ProbeCueTime
0,1,2,in_sequence,Low Words,0.857518,37,Yes,1,Responded,"[[1303746.2232548245], [1303747.22198163], [13...",...,"[[29, 25, 30]]","[[[wash.wav]], [[sure.wav]], [[youth.wav]]]",Low Words,29,wash.wav,0,D23,NaN,NaN,NaN
1,1,2,in_sequence,High Words,1.339604,37,Yes,1,Responded,"[[1303755.6904398678], [1303756.689051311], [1...",...,"[[2, 5, 8, 4, 7, 10, 3]]","[[[debt.wav]], [[lair.wav]], [[seek.wav]], [[g...",High Words,5,lair.wav,1,D23,NaN,NaN,NaN
2,1,2,in_sequence,High Non-Words,1.045657,37,Yes,1,Responded,"[[1303769.5315112157], [1303770.530119785], [1...",...,"[[19, 14, 18, 16, 20, 15, 17]]","[[[rihn.wav]], [[kehl.wav]], [[rehl.wav]], [[p...",High Non-Words,16,paon.wav,2,D23,NaN,NaN,NaN
3,1,1,out_of_sequence,High Words,1.497001,39,No,1,Responded,"[[1303783.0130984494], [1303784.0117098927], [...",...,"[[10, 2, 7, 9, 1, 8, 6, 5, 4]]","[[[witch.wav]], [[debt.wav]], [[pawn.wav]], [[...",Low Words,29,wash.wav,3,D23,NaN,NaN,NaN
4,1,1,out_of_sequence,Low Words,1.679783,39,No,1,Responded,"[1303798.9713094758, 1303799.969971005, 130380...",...,"[30, 24, 25, 29, 21]","[[youth.wav], [noun.wav], [sure.wav], [wash.wa...",High Words,9,soar.wav,4,D23,NaN,NaN,NaN


In [450]:
final_ieeg.isnull().sum()

block                       0
ProbeType                   0
ProbeTypeName               0
StimlusCategory             0
ReactionTime                0
KeyCode                     0
Resp                        0
RespCorrect                 0
Omission                  160
stimulusAudioStart          0
stimulusAlignedTrigger      0
probeAudioStart             0
probeAlignedTrigger         0
TriggerValue                0
stimulusSounds_idx          0
stimulusSounds_name         0
ProbeCategory               0
probeSound_idx              0
probeSound_name             0
Trial                       0
Subject                     0
ListenCueTime             953
MaintenancePeriodTime     953
ProbeCueTime              953
dtype: int64

## EEG dataset

In [456]:
data_eeg = eeg['Subject'] # contains trials for each of the subjects
print('Number of subjects in EEG dataset is', data_eeg.shape[1])

Number of subjects in EEG dataset is 29


In [470]:
subjects_eeg = [] # subject numbers
for i in range(0, data_eeg.shape[1]):
  name = data_eeg[0, i]['Name'][0][0][0]
  subjects_eeg.append(name)

In [487]:
columns_eeg = data_eeg[0, 0]['trialInfo'][0, 0].dtype.names # name of the variables from trial info
final_eeg = pd.DataFrame(columns=columns_eeg)
final_eeg['Trial'] = np.nan
final_eeg['Subject'] = np.nan
for subj in range(0, data_eeg.shape[1]): # for each subject
  print(subj)
  subj_df = get_to_df(data_eeg[0, subj]['trialInfo'][0, 0])
  subj_df['Subject'] = subjects_eeg[subj]
  final_eeg = pd.concat([final_eeg, subj_df], axis = 0)

0
1
2
3
4


KeyboardInterrupt: ignored

In [ ]:
final_eeg.head()

In [ ]:
final_eeg.isnull().sum()